## Pizza Classifier
Determines whether an image contains a pizza<br>
Created for CS50<br>
Dataset: https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza<br>
Last update: 12/23/23

In [35]:
import cv2
import cupy as np
from importlib import reload
from os import listdir
from sandbox import model, layers, activations, costs, utils

In [36]:
# Load altered pizza dataset
dataset_path = 'dataset\\altered\\'
datasets = [
    np.array([
        cv2.imread(dataset_path + dir + img).flatten()
        for img in listdir(dataset_path + dir)
    ])
    for dir in ['train\\pizza\\', 'train\\not_pizza\\', 'test\\pizza\\', 'test\\not_pizza\\']
]

# Create training and testing data
train_x = np.concatenate((datasets[0], datasets[1]), axis=0) / 255
train_y = np.concatenate((np.ones((datasets[0].shape[0], 1)), np.zeros((datasets[1].shape[0], 1))), axis=0)

test_x = np.concatenate((datasets[2], datasets[3]), axis=0) / 255
test_y = np.concatenate((np.ones((datasets[2].shape[0], 1)), np.zeros((datasets[3].shape[0], 1))), axis=0)

In [37]:
reload(model)

# Create model
pizza = model.Model(cuda=True)
pizza.add(layers.Dense(units=32, activation=activations.ReLU()))
pizza.add(layers.Dense(units=16, activation=activations.ReLU()))
pizza.add(layers.Dense(units=8, activation=activations.ReLU()))
pizza.add(layers.Dense(units=1, activation=activations.Sigmoid()))
pizza.configure(input_size=train_x.shape[1], cost_type=costs.BinaryCrossentropy())

# Train model
pizza.train(train_x, train_y, learning_rate=0.01, epochs=2500, verbose=True)

Cost on epoch 250: 0.59232
Cost on epoch 500: 0.58441
Cost on epoch 750: 0.54866
Cost on epoch 1000: 0.49703
Cost on epoch 1250: 0.49324
Cost on epoch 1500: 0.46957
Cost on epoch 1750: 0.46458
Cost on epoch 2000: 0.42426
Cost on epoch 2250: 0.44003
Cost on epoch 2500: 0.48667


In [38]:
# Print model summary
pizza.summary()

# Assess model accuracy
pred_train = utils.binary_round(pizza.predict(train_x)) # Get model accuracy on training data
print('\nTraining Accuracy: ' + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = utils.binary_round(pizza.predict(test_x)) # Get model accuracy on testing data
print('Testing Accuracy: ' + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   393248   |
|   Dense    |    528     |
|   Dense    |    136     |
|   Dense    |     9      |
+------------+------------+
Total parameters: 393921

Training Accuracy: 0.79172
Testing Accuracy: 0.68805
